In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from itertools import combinations, product
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [4]:
def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  try:
    html = browser.page_source
  except:
    print('something wrong with ' + url)
    html = None
  return html

In [5]:
df_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\first_iteration\words_in_clusters_manual_cleaning.csv"
df_sentences = pd.read_csv(df_path)

In [6]:
df_sentences.set_index(['Unnamed: 0'], inplace=True)

In [7]:
df_sentences

,old_list,new_list,comments
Unnamed: 0,,,
-1,company solar cryptocurrency,solar cryptocurrency,NaN
0,fusion nuclear plasma power reactors reactor f...,fusion nuclear plasma power reactor fission fuel,NaN
1,brain lobe body jess cells function parts driv...,brain lobe body cells function parts driver,NaN
2,50 super space computing,super space computing,NaN
3,3d emerging printers printing list 10 reality ...,3d emerging printers printing reality ai trends,NaN
4,iot internet things devices connected billion ...,iot internet of things devices connected smart...,NaN
5,following reasonable 2031 2068 vaguely thingsa...,NaN,empty content
6,predictions cars electric cp group futurists t...,predictions cars electric cp_group futurists ...,NaN
7,students communication issue letter cover busi...,students communication issue business informa...,NaN


In [8]:
df_sentences = df_sentences['new_list'].to_frame()

In [9]:
df_sentences

,new_list
Unnamed: 0,
-1,solar cryptocurrency
0,fusion nuclear plasma power reactor fission fuel
1,brain lobe body cells function parts driver
2,super space computing
3,3d emerging printers printing reality ai trends
4,iot internet of things devices connected smart...
5,NaN
6,predictions cars electric cp_group futurists ...
7,students communication issue business informa...


In [10]:
df_sentences.dropna(axis=0, inplace=True)

In [11]:
df_sentences

,new_list
Unnamed: 0,
-1,solar cryptocurrency
0,fusion nuclear plasma power reactor fission fuel
1,brain lobe body cells function parts driver
2,super space computing
3,3d emerging printers printing reality ai trends
4,iot internet of things devices connected smart...
6,predictions cars electric cp_group futurists ...
7,students communication issue business informa...
8,computers desktops road wrist keyboard


In [13]:
search_link = 'https://www.quora.com/search?q='

#create search links
all_search_links = []
for i, row in df_sentences.iterrows():
    sentence_words = row.iloc[0].split()
    # sentence = row.iloc[0].replace(' ','%20')
 
    #make all possible combinations with words 
    for s in range(len(sentence_words), 1+len(sentence_words)): 
     	for item in combinations(sentence_words, s): 
 	        pr = product(item, repeat=len(item))
            
            #remove first and last elements which are repetition of the same word
            pr = p[len(item):-len(item)]
            all_search_links.append(search_link+'%20'.join(pr))


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 14)

In [41]:
all_search_links

['https://www.quora.com/search?q=solar%20cryptocurrency',
 'https://www.quora.com/search?q=fusion%20nuclear%20plasma%20power%20reactor%20fission%20fuel',
 'https://www.quora.com/search?q=brain%20lobe%20body%20cells%20function%20parts%20driver',
 'https://www.quora.com/search?q=super%20space%20computing',
 'https://www.quora.com/search?q=3d%20emerging%20printers%20printing%20reality%20ai%20trends',
 'https://www.quora.com/search?q=iot%20internet%20of%20things%20devices%20connected%20smart%20network%20home%20embedded',
 'https://www.quora.com/search?q=predictions%20cars%20electric%20cp_group%20futurists%20prototype',
 'https://www.quora.com/search?q=students%20communication%20issue%20business%20information',
 'https://www.quora.com/search?q=computers%20desktops%20road%20wrist%20keyboard',
 'https://www.quora.com/search?q=ai%20programming%20cars%20selfflying%20artificial%20machine%20time%20robots%20learning',
 'https://www.quora.com/search?q=kaspersky%20future%20predict%20machines%20answe

In [36]:
len(all_search_links)

16

In [39]:
class Extract_href_urls():
    def __init__(self, links):
        self.links = links
        self.all_box_links = []
        self.all_indexes = []
        self.get_question_links()
        self.all_hrefs = []        
        self.get_hrefs()
          
    def get_question_links(self):    
        for j, link in enumerate(self.links):
            url_html = get_html_from_url(link)
            if url_html is None:
                continue
            soup = BeautifulSoup(url_html)
            #find all boxes of groups
            self.all_box_l = soup.find_all('a', {'class':"q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI"})
            self.all_indexes.extend([df_sentences.index[j]]*len(self.all_box_l))
            self.all_box_links.extend(self.all_box_l)

    @staticmethod
    def get_url_tag(box):
        #By find_element not working in selenium using string parsing instead
        url=None
        box = str(box)
        split_box = box.split()
        for word in split_box:
            if 'href' in word:
                url = word.strip('href=').strip('"')
                break
        return url
    
    def get_hrefs(self): 
        for box in self.all_box_links:
            link_hrefs = self.get_url_tag(box)
            self.all_hrefs.append(link_hrefs)

In [40]:
res1 = Extract_href_urls(all_search_links)

C:\Users\Ravit\AppData\Local\Temp\ipykernel_1360\1869657974.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
something wrong with https://www.quora.com/search?q=students%20communication%20issue%20%20business%20information
End of page reached
End of page reached
End of page reached


KeyboardInterrupt: 

In [ ]:
df_res = pd.DataFrame(res1.all_hrefs)

In [ ]:
df_res

,0
0,https://www.quora.com/What-would-happen-to-cry...
1,https://www.quora.com/What-solar-panels-would-...
2,https://www.quora.com/Is-Cryptocurrency-not-ge...
3,https://www.quora.com/Would-mining-cryptocurre...
4,https://www.quora.com/If-I-have-free-electrici...
...,...
383,https://www.quora.com/How-do-I-find-a-keyword-...
384,https://www.quora.com/What-does-the-revolution...
385,https://www.quora.com/Can-I-retrieve-deleted-p...
386,https://www.quora.com/What-is-cloud-computing-...


In [ ]:
df_res['cluster_index'] = res1.all_indexes

In [ ]:
df_res.columns = ['urls', 'cluster_index']
df_res = df_res[['cluster_index', 'urls']]

In [ ]:
df_res

,cluster_index,urls
0,-1,https://www.quora.com/What-would-happen-to-cry...
1,-1,https://www.quora.com/What-solar-panels-would-...
2,-1,https://www.quora.com/Is-Cryptocurrency-not-ge...
3,-1,https://www.quora.com/Would-mining-cryptocurre...
4,-1,https://www.quora.com/If-I-have-free-electrici...
...,...,...
383,13,https://www.quora.com/How-do-I-find-a-keyword-...
384,13,https://www.quora.com/What-does-the-revolution...
385,13,https://www.quora.com/Can-I-retrieve-deleted-p...
386,13,https://www.quora.com/What-is-cloud-computing-...


In [ ]:
save_path = r'C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\second_iteration\links_for_second_quora_scrap.csv'
df_res.to_csv(save_path, index=False)

In [ ]:
set(df_res['cluster_index'])

{-1, 1, 3, 12, 13}

In [ ]:
set(df_sentences.index) - set(df_res['cluster_index'])

{0, 2, 4, 6, 7, 8, 10, 11, 15, 16, 17}